# 数据结构与算法

## 将序列分解为单独的变量

In [1]:
data = ['ACME', 50, 91.1, (2020, 12, 21)]
# 任何序列（或可迭代的对象）都可以通过一个简单的赋值操作来分解为单独的变量。
# 要求变量的总数和结构要与序列相吻合
name, shares, price, date = data 
name

'ACME'

In [2]:
date

(2020, 12, 21)

In [3]:
name, shares, price, (year, mon, day) = data
year

2020

In [4]:
s = 'hello' # 只要对象是可迭代的就可以执行分解操作，包括字符串、文件、迭代器以及生成器
a, b, c, d, e = s
a

'h'

In [5]:
_, shares, price, _ = data # 选一个用不到的变量名，作为要丢弃的值的名称
shares

50

## 从任意长度的可迭代对象中分解元素

In [6]:
record = ['Dave', '773-555-1111', '773-555-1112', 'dave@example.com']
name, *phonenumbers, email = record # 使用 Python 的“*表达式”
email

'dave@example.com'

In [7]:
phonenumbers

['773-555-1111', '773-555-1112']

In [8]:
# *表达式 的语法在迭代一个变长的元组序列时尤其有用
records = [('foo', 1, 2), ('bar', 'hello'), ('foo', 3 ,4)]

def do_foo(x, y):
    print('foo', x, y)
    
def do_bar(s):
    print('bar', s)

for tag, *args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

foo 1 2
bar hello
foo 3 4


In [9]:
# 和某些特定的字符串处理操作（例如拆分）相结合
line = 'nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false'
uname, *fields, homedir, sh = line.split(':')
fields

['*', '-2', '-2', 'Unprivileged User']

In [10]:
sh

'/usr/bin/false'

In [11]:
name, *_, (*_, day) = data # 丢弃多个值
day

21

In [12]:
items = [1, 2, 4, 6, 8] # 将列表分解为头部和尾部
head, *tail = items
tail

[2, 4, 6, 8]

## 保存最后N个元素

In [ ]:
# 使用collections.deque保存有限的历史记录
# 对一系列文本行做简单的文本匹配操作，当发现有匹配时就输出当前的匹配行以及最后检查过的N行文本
from collections import deque

def search(lines, pattern, history = 5):
    previous_lines = deque(maxlen = history) # 创建了一个固定长度的队列
    for line in lines:
        if pattern in lines:
            yield line, previous_lines 
            # yield 的作用就是把一个函数变成一个 generator，带有 yield 的函数不再是一个普通函数，Python 解释器会将其视为一个 generator
            # 每执行到一个 yield 语句就会中断，并返回一个迭代值，下次执行时从 yield 的下一个语句继续执行
        previous_lines.append(line)
        
if __name__ == '__main__':
    with open('somefile.txt') as f:
        for line, prevlines in search(f, 'python', 5):
            for pline in prevlines:
                print(pline, end = '')
                print('-'*20)

# python中yield的用法详解 https://blog.csdn.net/mieleizhi0522/article/details/82142856

In [14]:
from collections import deque
q = deque()
q.append(1)
q.append(2)
q

deque([1, 2])

In [15]:
q.appendleft(4)
q

deque([4, 1, 2])

In [16]:
q.pop()

2

In [17]:
q.popleft() # 从队列两端添加或者弹出元素的复杂度都是O(1)，从列表的头部插入或移除元素的复杂度是O(n)

4

In [18]:
q

deque([1])

## 找到最大或最小的N个元素

In [19]:
# heapq 模块中的nlargest()和nsmallest()函数,这两个函数可以接受一个参数key，从而允许在更加复杂的数据结构中使用
import heapq
nums = [1, 3, 2, 0, -1, 8, 5]
print(heapq.nlargest(3, nums))
print(heapq.nsmallest(2, nums))

[8, 5, 3]
[-1, 0]


In [21]:
import heapq

portfolio = [
   {'name': 'IBM', 'shares': 100, 'price': 91.1},
   {'name': 'AAPL', 'shares': 50, 'price': 543.22},
   {'name': 'FB', 'shares': 200, 'price': 21.09},
   {'name': 'HPQ', 'shares': 35, 'price': 31.75},
   {'name': 'YHOO', 'shares': 45, 'price': 16.35},
   {'name': 'ACME', 'shares': 75, 'price': 115.65}
]

cheap = heapq.nsmallest(3, portfolio, key=lambda s: s['price'])
expensive = heapq.nlargest(3, portfolio, key=lambda s: s['price'])

print(cheap)
print(expensive)

[{'name': 'YHOO', 'shares': 45, 'price': 16.35}, {'name': 'FB', 'shares': 200, 'price': 21.09}, {'name': 'HPQ', 'shares': 35, 'price': 31.75}]
[{'name': 'AAPL', 'shares': 50, 'price': 543.22}, {'name': 'ACME', 'shares': 75, 'price': 115.65}, {'name': 'IBM', 'shares': 100, 'price': 91.1}]


## 实现优先级队列

In [4]:
import heapq # https://docs.python.org/zh-cn/3/library/heapq.html

class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._index = 0 # 通过维护一个递增的索引，将相同优先级的元素以入队顺序排列
        
    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))
        self._index += 1
    
    def pop(self):
        return heapq.heappop(self._queue)[-1] # heappop()返回“最小”的元素，正常的堆排序是按从小到大排的，此处priority取负值

class Item:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return 'Item({!r})'.format(self.name)


q = PriorityQueue()
q.push(Item('foo'), 1)
q.push(Item('bar'), 5)
q.push(Item('spam'), 4)
q.push(Item('grok'), 1)
q.pop() # 第一次pop()操作返回的元素具有最高优先级，优先级相同的元素返回顺序同插入队列的顺序相同

Item('bar')

如果以元组(priority, item)的形式表示元素，那么只要优先级不同就可以比较。priority相同时比较操作会报错，引入额外的索引值，以(priority, index, item)建立元组可以避免这个问题。

## 在字典中将键映射到多个值上

In [ ]:
# 字典是一种关联容器，每个键都映射到一个值上。如果想让键映射到多个值（一键多值字典 multidict）,需要将这多个值保存到另一个如列表或集合中。
d = {
    'a': [1, 2, 3], 
    'b': [4, 5]
}
e = {
    'a': {1, 2, 3},
    'b': {4, 5}
}
# 保留元素插入顺序用列表，否则用集合

In [6]:
from collections import defaultdict # defaultdict 会初始化第一个值，只需关注添加元素即可

# dict =defaultdict( factory_function)
# defaultdict的作用是在于，当字典里的key不存在但被查找时，返回的不是keyError而是一个默认值
# 这个factory_function可以是list、set、str等等，作用是当key不存在时，返回的是工厂函数的默认值，
# 比如list对应[ ]，str对应的是空字符串，set对应set( )，int对应0

d = defaultdict(list) 
d['a'].append(1)
d['a'].append(2)
d['b'].append(4)
print(d['c']) # []

e = defaultdict(set)
e['a'].add(1)
e['a'].add(2)
e['b'].add(4)
print(e['c']) # set()

# defaultdict会自动创建字典表项以待稍后的访问（即使这些表项当前在字典中还没有找到）。如果不想要这个功能，可以在普通字典上调用setdefault()取代
f = {} # 普通字典
f.setdefault('a', []).append(1)
f.setdefault('a', []).append(2)
f.setdefault('b', []).append(4)
# 每次调用setdefault()会创建一个初始值的新实例（例子中的空列表[])

[]
set()


In [ ]:
d = {}
for key, value in pairs:
    if key not in d:
        d[key] = [] # 需要对第一个值做初始化操作
    d[key].append(value)
    
e = defaultdict(list)
for key, value in pairs:
    e[key].append(value)

## 让字典保持有序

In [8]:
from collections import OrderedDict # 使用OrderedDict，当对字典做迭代时会严格按照元素的初始添加顺序进行

d = OrderedDict()
d['foo'] = 1
d['bar'] = 2
d['spam'] = 3

for key in d:
    print(key, d[key])

# foo 1
# bar 2
# spam 3

# 在构建一个映射结构后面需要对其做序列化或编码成另一种格式时可以使用OrderedDict
# 例如在进行JSON编码时精确控制各字段的顺序
import json

json.dumps(d) # '{"foo": 1, "bar": 2, "spam": 3}'

# OrderedDict内部维护了一个双向链表，根据元素的加入顺序排列键的位置。
# 第一个加入的元素被放置在链表末尾，接下来对已存在的键做重新赋值时不会改变键的顺序。
# 注意OrderedDict的大小是普通字典的2倍多，是由于它额外创建的链表所致。

foo 1
bar 2
spam 3


'{"foo": 1, "bar": 2, "spam": 3}'

## 与字典有关的计算问题

In [13]:
# 在一个字典上对股票名称和价格做了映射
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55
}

# 为了后续计算通常会使用zip()将字典的键和值反转过来
# zip() 函数用于将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组，然后返回由这些元组组成的对象
min_price = min(zip(prices.values(), prices.keys())) # (45.23, 'ACME')
max_price = max(zip(prices.values(), prices.keys())) # (612.78, 'AAPL')

prices_sorted = sorted(zip(prices.values(), prices.keys())) # 注意：zip()所创建迭代器的内容只能被消费一次

min(prices, key=lambda k: prices[k]) # 提供一个key参数传递给min()就能得到最小值所对应的键是什么

'ACME'

In [14]:
prices = {'AA' : 45.23, 'BB' : 45.23} # value值相同时使用key作为判定结果
min(zip(prices.values(), prices.keys())) # (45.23, 'AA')

(45.23, 'AA')

## 在两个字典中寻找相同点

In [18]:
a = {
    'x' : 1,
    'y' : 2,
    'z' : 3
}

b = {
    'w' : 10,
    'x' : 11,
    'y' : 2
}

# 寻找两个字典的相同点，只需通过keys()或items()执行常见的集合操作
# 字典的values()不支持集合操作

# Find keys in common
a.keys() & b.keys() # {'x', 'y'}

# Find keys in a that are not in b
a.keys() - b.keys() # {'z'}

# Find (key, value) pairs in common
a.items() & b.items() # {('y', 2)}

{('y', 2)}

In [ ]:
# 修改或过滤字典的内容
# make a new dictionary with certain keys removed
c = {key:a[key] for key in a.keys() - {'z', 'w'}}
# c is {'x': 1, 'y': 2}

## 从序列中移除重复项且保持元素间顺序不变